### Create Q and A Chatbot using Assistants API

In [17]:
import os
import json
import time
import openai


def show_json(obj):
    display(json.loads(obj.model_dump_json())) 

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

client = openai.OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
file_path = "..\\..\\data\\pdf\\test\\2308.00479.pdf"
user_prompt = "what's the tile of the paper?"

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open(file_path, "rb"),
  purpose='assistants'
)

# Add the file to the assistant
# You can attach a maximum of 20 files per Assistant, and they can be at most 512 MB each. 
# In addition, the size of all the files uploaded by your organization should not exceed 100GB. 
# You can request an increase in this storage limit using our help center.
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

# thread = client.beta.threads.create()

# message = client.beta.threads.messages.create(
#   thread_id=thread.id,
#   role="user",
#   content=user_prompt,
#   file_ids=[file.id]
# )

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": user_prompt,
      "file_ids": [file.id]
    }
  ]
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run = wait_on_run(run, thread)
messages = client.beta.threads.messages.list(thread_id=thread.id)

pretty_print(messages)
# messages.data[0].content[0].text.value

# Messages
assistant: The title of the paper is "Retrieval Augmented Generation and Representative Vector Summarization for Large Unstructured Textual Data in Medical Education".
user: what's the tile of the paper?



In [18]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [19]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    run = wait_on_run(run, thread)
    pretty_print(get_response(thread))

# create_thread_and_run("what's the tile of the paper?")

In [20]:
input_instructions = f"""I am a customer support chatbot to answer questions related to {file_path},
    you can type your questions, or type Bye to exit:
"""
while True:
    user_input = input(input_instructions)
    
    if "bye" in user_input.lower():
        print("Exiting the program. Goodbye!")
        break
    else:
        print("You typed:", user_input)
        create_thread_and_run(user_input)


You typed: list authors of the paper
# Messages
user: list authors of the paper
assistant: The authors of the paper are:

- S. S. Manathunga, Department of Pharmacology, University of Peradeniya, Sri Lanka (ssm123ssm@gmail.com)
- Y. A. Illangasekera, Department of Pharmacology, University of Peradeniya, Sri Lanka (yasi04@gmail.com)【7†source】.

You typed: Can you summarize the paper 
# Messages
user: Can you summarize the paper 
assistant: The paper titled "Retrieval Augmented Generation and Representative Vector Summarization for Large Unstructured Textual Data in Medical Education" discusses how Large Language Models (LLMs) are increasingly being utilized for content generation and chatbots. However, these models must be specifically aligned when applied to domain-specific tasks to avoid issues such as producing incorrect or harmful answers. A technique called Retrieval Augmented Generation (RAG) is proposed, which facilitates the easy attachment and manipulation of non-parametric kno